In [2]:
import random
import pickle

import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [12]:
#import the dataset and make it into a list
text_df = pd.read_csv("../dataset/fake_or_real_news.csv")
text = list(text_df.text.values)
joined_text = " ".join(text)

In [13]:
#splitting and resulting individual word in a list
partial_text = joined_text[:10000]
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(partial_text.lower())

In [14]:
#removing duplicate word
unique_tokens = np.unique(tokens)
unique_tokens_index = {token: idx for idx, token in enumerate(unique_tokens)}

In [15]:
n_words = 10
input_words = []
next_words = []

for i in range(len(tokens) - n_words):
    input_words.append(tokens[i: + n_words])
    next_words.append(tokens[i + n_words])

In [16]:
x = np.zeros((len(input_words), n_words, len(unique_tokens)), dtype=bool)
y = np.zeros((len(next_words), len(unique_tokens)), dtype=bool)

In [17]:
for i, words in enumerate(input_words):
    for j, word in enumerate(words):
        x[i, j, unique_tokens_index[word]] = 1
    y[i, unique_tokens_index[next_words[i]]] = 1

In [18]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_words, len(unique_tokens)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

c:\Users\vince\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [20]:
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(x, y, batch_size=128, epochs=10, shuffle=True).history

Epoch 1/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - accuracy: 0.0365 - loss: 6.2944
Epoch 2/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.0635 - loss: 5.8333
Epoch 3/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.0592 - loss: 5.7403
Epoch 4/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.0592 - loss: 5.7696
Epoch 5/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.0702 - loss: 5.7127
Epoch 6/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.0692 - loss: 5.6968
Epoch 7/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.0579 - loss: 5.7763
Epoch 8/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.0595 - loss: 5.7092
Epoch 9/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.0600 - loss: 5.7303
Epoch 10/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.0585 - loss: 5.7088


In [21]:
history = model.fit(x, y, batch_size=128, epochs=5, shuffle=True).history

Epoch 1/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.0623 - loss: 5.7658
Epoch 2/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.0576 - loss: 5.7166
Epoch 3/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.0526 - loss: 5.7502
Epoch 4/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.0665 - loss: 5.6730
Epoch 5/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.0659 - loss: 5.6551


In [22]:
model.save("text_gen_model2.h5")
with open("history2.p", "wb") as f:
    pickle.dump(history, f)

In [7]:
model = load_model("../model/text_gen_model2.h5")
history = pickle.load(open("../model/history2.p", "rb"))

In [8]:
def predict_next_word(input_text, n_best):
    input_text = input_text.lower()
    x = np.zeros((1, n_words, len(unique_tokens)))
    for i, word in enumerate(input_text.split()):
        x[0, i, unique_tokens_index[word]] = 1
    
    predictions = model.predict(x)[0]
    return np.argpartition(predictions, -n_best)[-n_best:]

In [19]:
input_text = "You are so"
input_words = input_text.lower().split()
for word in input_words:
    if word not in unique_tokens_index:
        print(f"Word '{word}' not in unique_tokens_index")
possible = predict_next_word(input_text, 5)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 824ms/step


In [20]:
for idx in possible:
    print(unique_tokens[idx])

chilly
illegal
paul
role
those


In [22]:
def generate_text(input_text, n_words, creativity=3):
    word_sequence = input_text.split()
    current = 0
    for _ in range(n_words):
        sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current+n_words])
        try:
            choice = unique_tokens[random.choice(predict_next_word(sub_sequence, creativity))]
        except:
            choice  = random.choice(unique_tokens)
        word_sequence.append(choice)
        current += 1
    return " ".join(word_sequence)

In [23]:
generate_text("I will have to look into this thing because I", 100, 10)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━

'I will have to look into this thing because I prominent times hard particularly smart sexual tape spinelessness display hoover one days attack conspiracy conspiracy attack new unprecedented believes letter twitter letter united be abcpolitics focused age james james gasoline james elect longer elect countless longer world center world center to train want waking panicked smell career cycle can born even thing aboutface down start edgar your fireworks appeaser anthony t age often any any quickly surreal bathroom quickly center couldn quickly want surreal want want decided procedure election house victory where results where asked focused were may futures results digg investigation recalls investigation lambasting pass admits than digg as'